# 感情分析に基づく分類

## パッケージのインストール

In [ ]:
!pip install oseti

## 基本的な使い方

In [ ]:
import oseti

analyzer = oseti.Analyzer()
print(analyzer.analyze_detail('天国で待ってる。'))
print(analyzer.analyze_detail('遅刻したけど楽しかったし嬉しかった。すごく充実した！'))

In [ ]:
from collections import Counter
import oseti

filename = 'text/kageotoko.corpus.txt'
analyzer = oseti.Analyzer()

with open(filename, 'r', encoding='UTF-8') as f:
    hist = Counter(
        element['score']
        for line in f
        for element in analyzer.analyze_detail(line))

print('score count')
print('-----------')
for k in sorted(hist.keys()):
    print('{:>5.02f} {}'.format(k, hist[k]))

# トピックモデルによるクラスタリング

## パッケージのインストール

In [ ]:
!pip install gensim pandas openpyxl
!pip install ginza ja-ginza

## 解析対象データをダウンロード

In [ ]:
!mkdir -p sisyou_db

for year in ['h18', 'h19', 'h20', 'h21', 'h22', 'h23', 'h24']:
    for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        !wget -P sisyou_db -c https://anzeninfo.mhlw.go.jp/anzen/shisyo_xls/sisyou_db_{year}_{month}.xls

for year in ['h25', 'h26', 'h27', 'h28', 'h29']:
    for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        !wget -P sisyou_db -c https://anzeninfo.mhlw.go.jp/anzen/shisyo_xls/sisyou_db_{year}_{month}.xlsx

## サンプルコードでは平成29年1月分のデータしか利用しませんが、
## データとしては平成18年〜平成29年の12年分のデータをダウンロードします。
## ぜひご自身でも分析してみてください、

## 基本的な使い方

In [ ]:
import pandas
df=pandas.read_excel('sisyou_db/sisyou_db_h29_01.xlsx')
print(df['災害状況'])

In [ ]:
import logging
import spacy
import pandas as pd 

# 1. 学習中の状況を出力するフォーマットの指定
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.INFO)

# 2. 解析器の初期化
nlp = spacy.load("ja_ginza")

# 3. ドキュメントを読み込んで単語分割する
df=pd.read_excel('sisyou_db/sisyou_db_h29_01.xlsx')
docs=[]
for text in df['災害状況']:
    if type(text) != str: continue
    doc=[token.lemma_ for token in nlp(text)]
    docs.append(doc)

#
# 4. 極端に頻度が低い語や高い語を除外する
#
from gensim.corpora import Dictionary

# 4a. 辞書の作成
dictionary = Dictionary(docs)

# 4b. 出現が20文書に満たない単語と50%以上の文書に出現する単語を極端とみなして除外する
dictionary.filter_extremes(no_below=20, no_above=0.5)

# 4c. 上記のように定められた語彙で、文書単語行列(Bag-of-words表現)を求める
corpus = [dictionary.doc2bow(doc) for doc in docs]

print(f'Number of unique tokens: {len(dictionary)}')
print(f'Number of documents: {len(corpus)}')

#
# 5. バイグラムの計算
#
from gensim.models import Phrases

# 5a. 文書単語行列にバイグラムを加える (出現頻度20以上のもの)
phrase_model = Phrases(docs, min_count=20)
for idx, doc in enumerate(docs):
    # _が含まれるトークンはバイグラム(あるいはそれ以上)なので、追加する
    bigram_tokens = [token for token in phrase_model[doc] if '_' in token]
    docs[idx].extend(bigram_tokens)

#            
# 6. LDAモデル
#
from gensim.models import LdaModel

num_topics = 10

temp = dictionary[0]  # 辞書をメモリに読み込むための処理

# 6a. LDAモデルの計算
model = LdaModel(
    corpus=corpus,
    id2word=dictionary.id2token,
    chunksize=2000,
    alpha='auto',
    eta='auto',
    iterations=400,
    num_topics=num_topics,
    passes=20,
)

# 6b. LDAモデルによって得られるトピックの抽出
top_topics = model.top_topics(corpus) 
coherences = [coherence for topic_words, coherence in top_topics]

# 6c. 評価指標コヒーレンスを求める
avg_topic_coherence = sum(coherences) / num_topics
print(f'Average topic coherence: {avg_topic_coherence:.4f}.')